In [1]:
import sys
import re
import time
import datetime
import pandas as pd
import func
# inline plot
%matplotlib inline

/Users/albarron/anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
all_trains = pd.read_csv("data/weather-trains_2015-06-08 16:57:272016-01-25 23:42:14.csv",sep='\t',error_bad_lines=False)
delayed = pd.read_csv("data/merged_delay_2016-03-27-02:02:47.csv",sep='\t',error_bad_lines=False)
del delayed['Unnamed: 0']
del all_trains['Unnamed: 0']

In [3]:
print all_trains.shape
print delayed.shape

(21021, 12)


In [4]:
del all_trains['time_pa']
all_trains.head()

,train_id,stop_pa,t_northbound,timestamp,temp,precipiation,visability,windspeed,humidity,cloudcover
0,102,1,0,2015-06-08 05:53:00,72.97875,0,10,6.68125,0.62875,0.09875
1,104,1,0,2015-06-08 06:23:00,72.97875,0,10,6.68125,0.62875,0.09875
2,206,1,0,2015-06-08 06:59:00,72.97875,0,10,6.68125,0.62875,0.09875
3,208,1,0,2015-06-08 07:20:00,72.97875,0,10,6.68125,0.62875,0.09875
4,210,1,0,2015-06-08 07:28:00,72.97875,0,10,6.68125,0.62875,0.09875


In [6]:
delayed.head()

,day_of_week,day_of_month,month,time_of_day,topic_train,t_bullet,t_limited,t_northbound,timestamp,train_id_x,...,is_mechanical,is_customer,is_event,id,temp,precipiation,visability,windspeed,humidity,cloudcover
0,0,25,1,error,'155',0,0,1,2016-01-25 23:42:14,155,...,0,0,0,691768068385718275,52.890417,0.000000,10.000000,3.979583,0.811667,0.1555
1,4,22,1,error,'151',0,0,1,2016-01-22 22:48:57,151,...,0,0,0,690667494906814464,59.498750,0.003975,8.580000,9.688750,0.820833,0.2325
2,2,20,1,error,'151',0,0,1,2016-01-20 22:22:55,151,...,0,0,0,689936168893329408,53.795000,0.000000,8.801667,3.730833,0.877917,0.1080
3,1,19,1,rush_evening,'138',0,0,0,2016-01-19 18:08:25,138,...,0,0,0,689509733640732672,57.295833,0.016792,7.550417,7.709583,0.869167,0.2200
4,1,19,1,rush_evening,"'216', '225', '329'",0,1,0,2016-01-19 16:32:04,216,...,0,0,0,689485484838416388,57.295833,0.016792,7.550417,7.709583,0.869167,0.2200


In [58]:
delayed.columns.values

array(['day_of_week', 'day_of_month', 'month', 'time_of_day',
       'topic_train', 't_bullet', 't_limited', 't_northbound', 'timestamp',
       'train_id_x', 'tweet_id', 'uid', 'is_delay', 'delay_minor',
       'delay_med', 'delay_major', 'delay_catastrophic', 'is_backlog',
       'is_canceled', 'is_passing', 'is_accident', 'is_medical',
       'is_mechanical', 'is_customer', 'is_event', 'id', 'temp',
       'precipiation', 'visability', 'windspeed', 'humidity', 'cloudcover'], dtype=object)

In [ ]:
merged = all_trains.merge(atw,left_on='time_pa',right_on='timestamp',how='outer')
print merged.shape
merged.head(5)

In [ ]:
# all_trains.merge(delayed,left_on=['train_id'])

In [7]:
all_trains['timestamp'] = pd.to_datetime(all_trains['timestamp'],format="%Y-%m-%d %H:%M")

In [38]:
delayed['timestamp'] = pd.to_datetime(all_trains['timestamp'],format="%Y-%m-%d %H:%M")

In [40]:
max_date = all_trains['timestamp'].max()
min_date = all_trains['timestamp'].min()
print max_date
print min_date

2016-01-24 23:38:00
2015-06-08 05:01:00


In [64]:
def populate_all_trains(t_row):
    t=t_row['timestamp']
    d = delayed[(delayed['timestamp'] > datetime.datetime(t.year,t.month,t.day)) 
           & (delayed['timestamp'] < datetime.datetime(t.year,t.month,t.day+1))]

    return d.shape
#     return pd.Series([
#         d.t_bullet,
#         d.t_limited,
#         d.tweet_id,
#         d.is_delay,
#         d.delay_minor,
#         d.delay_med,
#         d.delay_major,
#         d.delay_catastropic,
#         d.is_backlog,
#         d.is_canceled,
#         d.is_passing,
#         d.is_accident,
#         d.is_medical,
#         d.is_mechanical,
#         d.is_customer,
#         d.is_event
#     ])

In [65]:
all_trains.apply(lambda x:populate_all_trains(x),axis=1)

ValueError: ('day is out of range for month', u'occurred at index 2002')

In [57]:
# # Stolen here: http://stackoverflow.com/questions/1060279/iterating-through-a-range-of-dates-in-python
# def daterange(start_date, end_date):
#     for n in range(int ((end_date - start_date).days)):
#         yield start_date + datetime.timedelta(n)

# for cursor_day in daterange(min_date, min_date+datetime.timedelta(days=5) ): #max_date
#     for train in all_trains[(all_trains['timestamp'] > cursor_day) 
#            & (all_trains['timestamp'] < cursor_day+datetime.timedelta(days=1))]:
#         print train
# #         print delayed[(delayed['timestamp'] > cursor_day) 
# #             & (delayed['timestamp'] < cursor_day+datetime.timedelta(days=1))
# #             & (delayed['train_id_x'] == train['train_id'])]

train_id
stop_pa
t_northbound
timestamp
temp
precipiation
visability
windspeed
humidity
cloudcover
train_id
stop_pa
t_northbound
timestamp
temp
precipiation
visability
windspeed
humidity
cloudcover
train_id
stop_pa
t_northbound
timestamp
temp
precipiation
visability
windspeed
humidity
cloudcover
train_id
stop_pa
t_northbound
timestamp
temp
precipiation
visability
windspeed
humidity
cloudcover
train_id
stop_pa
t_northbound
timestamp
temp
precipiation
visability
windspeed
humidity
cloudcover


In [37]:
all_trains[(all_trains['timestamp'] > datetime.datetime(2015,6,8)) 
           & (all_trains['timestamp'] < datetime.datetime(2015,6,10))].shape

(182, 10)

In [25]:
all_trains[all_trains['timestamp'] > datetime.date(year=2016,month=6,day=8) &
           all_trains['timestamp'] < datetime.date(year=2016,month=6,day=9)]


TypeError: cannot compare a dtyped [datetime64[ns]] array with a scalar of type [bool]